In [ ]:
%%time
%%capture
!pip install transformers
#!pip install install pytorch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 cudatoolkit=11.2 -c pytorch

In [ ]:
!nvidia-smi
"""
VERSION = "20200220" #@param ["20200220","nightly", "xrt==1.15.0"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py 
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)"""

In [ ]:
import os
import io
import requests
import numpy as np
import pandas as pd
import re
import zipfile
import random
import time
import csv
import datetime
from itertools import compress
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer

import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler

from IPython.display import clear_output

print(f"PyTorch version: {torch.__version__}")

In [ ]:
import wandb
!wandb login "4c7a6eeec385556acc3eb3e77d3e7e111095da18"

In [ ]:
"""import torch_xla
import torch_xla.core.xla_model as xm
dev = xm.xla_device()
t1 = torch.ones(3, 3, device = dev)
print(t1)"""

In [ ]:
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dev

In [ ]:
DEBUG           = False

INPUT_DIR       = 'articles'

USE_APEX        = True
APEX_OPT_LEVEL  = 'O1'

MODEL           = "gpt2-medium" # "chrisliu298/arxiv_ai_gpt2" #'gpt2' #{gpt2, gpt2-medium, gpt2-large, gpt2-xl}

UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training

SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",                    
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}
                    
MAXLEN          = 256  #{768, 1024, 1280, 1600}

TRAIN_SIZE      = 0.9

if USE_APEX:
    TRAIN_BATCHSIZE = 8
    BATCH_UPDATE    = 16
else:
    TRAIN_BATCHSIZE = 2
    BATCH_UPDATE    = 32

EPOCHS          = 4
LR              = 5e-4
EPS             = 1e-8
WARMUP_STEPS    = 1e2

SEED            = 2020

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [ ]:

!pip3 install kaggle

# authenticate using public api token which can found from your kaggle profile page
%env KAGGLE_USERNAME=nayannae17b010
%env KAGGLE_KEY=f81b5ac285a14478bf294bd58c485a8f

# to download on drive 
# 1) mount drive using the option of the left tab
# 2) %cd drive/MyDrive/ 
!kaggle competitions download -c dl-hack-track-2-nlp
!unzip "./dl-hack-track-2-nlp.zip"

In [ ]:
columns = ['title',
           'abstract']
df = pd.read_csv("./train.csv", sep=",")
print(len(df))
df.head()

In [ ]:
"""word_count = 0
for i in range(1,len(df)) : word_count+= len(df.iloc[i]["abstract"].split(" "))
word_count/(len(df)-1)"""
df['abstract_length'] = df['abstract'].apply(lambda x : len(x.split(" ")))
print(df['abstract_length'].describe())
df.drop('abstract_length',axis=1,inplace=True)
#sum(df['abstract_length']>256)/146381

In [ ]:
%%time

data = dict()
"""   
for root, dirs, files in os.walk(INPUT_DIR, topdown=True):
    t0 = time.time()

    for i, f in enumerate(files):
        #id, category, title, keywords, text
        id = int(f[:-4])        
        tmp = df[['CATEGORY', 'TITLE']][df.ID==id].values
        category, title = tmp[0][0], tmp[0][1]

        with open(f'{INPUT_DIR}/{f}', "r") as infile:
            text = infile.read()
        
        data[id] = [title, text]

        if i%1000==0 and i>0:
            clear_output(wait=True)
            print(f"({os.getpid()}) Items processed: {i :,}/{len(files):,}; {(time.time()-t0)/60 :.1f} minutes")

            if DEBUG:
                break

print(f"Number of articles: {len(data) :,}")"""
for i in range(len(df)):
    data[i] = [df.iloc[i]['title'],df.iloc[i]['abstract']]

In [ ]:
# !gdown --id 1C1WWvnt2egzhRmVXMSJhARz5GOLDGzMC
# !cp "/content/keywords.csv.zip" "/content/drive/MyDrive/Colab Notebooks/dl_hack/data/"
# !unzip "/content/keywords.csv.zip" -d "/content/drive/MyDrive/Colab Notebooks/dl_hack/data/"
# !head "/content/drive/MyDrive/Colab Notebooks/dl_hack/data/keywords.csv"

In [ ]:
class myDataset(Dataset):

    def __init__(self, data, tokenizer, randomize=True):

        title, text = [], [] # , keywords = [],
        for k, v in data.items():
            title.append(v[0])
            text.append(v[1])
            # keywords.append(v[2])

        self.randomize = randomize
        self.tokenizer = tokenizer 
        self.title     = title
        self.text      = text
        # self.keywords  = keywords  

    #---------------------------------------------#

    @staticmethod
    def join_keywords(keywords, randomize=True):
        N = len(keywords)

        #random sampling and shuffle
        if randomize: 
            M = random.choice(range(N+1))
            keywords = keywords[:M]
            random.shuffle(keywords)

        return ','.join(keywords)

    #---------------------------------------------#

    def __len__(self):
        return len(self.text)

    #---------------------------------------------#
    
    def __getitem__(self, i):
        # keywords = self.keywords[i].copy()
        # kw = self.join_keywords(keywords, self.randomize)
        
        input = SPECIAL_TOKENS['bos_token'] + " " + self.title[i].strip() + " " + SPECIAL_TOKENS['sep_token'] + " " + self.text[i].strip() + " " + SPECIAL_TOKENS['eos_token'] # kw + SPECIAL_TOKENS['sep_token'] + \

        encodings_dict = tokenizer(input,                                   
                                   truncation=True, 
                                   max_length=MAXLEN, 
                                   padding="max_length")   
        
        input_ids = encodings_dict['input_ids']
        attention_mask = encodings_dict['attention_mask']
        
        return {'label': torch.tensor(input_ids,device=dev),
                'input_ids': torch.tensor(input_ids,device=dev), 
                'attention_mask': torch.tensor(attention_mask,device=dev)}

In [ ]:
def split_data(data, S=TRAIN_SIZE):
    # Shuffle ids
    ids = list(data.keys())
    random.shuffle(ids)

    # Split into training and validation sets    
    train_size = int(S * len(data))

    train_ids = ids[:train_size]
    val_ids = ids[train_size:]

    train_data = dict()
    for id in train_ids:
        train_data[id] = data[id]

    val_data = dict()
    for id in val_ids:
        val_data[id] = data[id]

    return train_data, val_data

In [ ]:
def get_tokenier(special_tokens=None):
    tokenizer = AutoTokenizer.from_pretrained(MODEL) #GPT2Tokenizer

    if special_tokens:
        tokenizer.add_special_tokens(special_tokens)
        print("Special tokens added")
    return tokenizer

def get_model(tokenizer, special_tokens=None, load_model_path=None):

    #GPT2LMHeadModel
    if special_tokens:
        config = AutoConfig.from_pretrained(MODEL, 
                                            bos_token_id=tokenizer.bos_token_id,
                                            eos_token_id=tokenizer.eos_token_id,
                                            sep_token_id=tokenizer.sep_token_id,
                                            pad_token_id=tokenizer.pad_token_id,
                                            output_hidden_states=False)
    else: 
        config = AutoConfig.from_pretrained(MODEL,                                     
                                            pad_token_id=tokenizer.eos_token_id,
                                            output_hidden_states=False)    

    #----------------------------------------------------------------#
    model = AutoModelForPreTraining.from_pretrained(MODEL, config=config)

    if special_tokens:
        #Special tokens added, model needs to be resized accordingly
        model.resize_token_embeddings(len(tokenizer))

    if load_model_path:
        model.load_state_dict(torch.load(load_model_path))

    model.cuda()
    return model

In [ ]:
%%time

#with tpu_strategy.scope():
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                special_tokens=SPECIAL_TOKENS,
                #load_model_path='./model_v2/pytorch_model.bin'
                )

In [ ]:
# - Freeze selective layers:
# - Freeze all layers except last n:
for parameter in model.parameters():
    parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):        
    #Only un-freeze the last n transformer blocks
    if i+1 > 12 - UNFREEZE_LAST_N:
        for parameter in m.parameters():
            parameter.requires_grad = True 

for parameter in model.transformer.ln_f.parameters():        
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():
    parameter.requires_grad = True

In [ ]:
train_data, val_data = split_data(data)

train_dataset = myDataset(train_data, tokenizer)
val_dataset = myDataset(val_data, tokenizer, randomize=False)

f'There are {len(train_dataset) :,} samples for training, and {len(val_dataset) :,} samples for validation testing'

In [ ]:
!mkdir "./model_med/"

In [ ]:
%%time

training_args = TrainingArguments(
    output_dir="./model_med/",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=TRAIN_BATCHSIZE,
    per_device_eval_batch_size=TRAIN_BATCHSIZE,
    gradient_accumulation_steps=BATCH_UPDATE,
    evaluation_strategy="steps",
    fp16=True,
    fp16_opt_level=APEX_OPT_LEVEL,
    warmup_steps=WARMUP_STEPS,    
    learning_rate=LR,
    adam_epsilon=EPS,
    weight_decay=0.01,        
    save_total_limit=5,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,   
)

#---------------------------------------------------#
trainer = Trainer(
    model=model,
    args=training_args,    
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

#---------------------------------------------------#
trainer.train()
trainer.save_model()    

In [ ]:
import time
df_test = pd.read_csv("./test.csv")
df_test.head()
abstracts = []
"""
device = torch.device("cuda")
model.to(device)
"""
tokenizer.padding_side = 'left'
model.eval();
with open("./predictions.csv",'w') as csv_file:
    
    csv_writer = csv.writer(csv_file,delimiter=',')
    csv_writer.writerow(['title','abstract'])
    start = time.time()
    for i in range(0,len(df_test),TRAIN_BATCHSIZE):
        print(i,time.time()-start)
        start=time.time()
        titles = []
        prompts = []
        for j in range(i,min(i+TRAIN_BATCHSIZE,len(df_test))):
            title = df_test.iloc[j]['title'].strip()
            titles.append(title)
            prompt = SPECIAL_TOKENS['bos_token'] + " " + title + " " + SPECIAL_TOKENS['sep_token'] + " "
            prompts.append(prompt)

        encoded_prompt_dict = tokenizer.batch_encode_plus(prompts, return_tensors="pt", pad_to_max_length=True)
        encoded_prompt = encoded_prompt_dict['input_ids'].to(dev)
        encoded_mask = encoded_prompt_dict['attention_mask'].to(dev)
        
        ops = model.generate(encoded_prompt,
                                       attention_mask = encoded_mask,
                                        do_sample=True,   
                                        min_length=100, 
                                        max_length=MAXLEN,
                                        top_k=15,                                 
                                        top_p=0.9,        
                                        temperature=0.8,
                                        repetition_penalty=2.0,
                                        num_return_sequences=1,
                                        length_penalty=1.0
                                        )
        for title,op in zip(titles,ops):
            text = tokenizer.decode(op, skip_special_tokens=True)
            abstract = text[len(title):]
            csv_writer.writerow([title,abstract.strip()])
!python3 "../input/submission-script/submit_track2.py" "./predictions.csv"
#torch.cuda.empty_cache()

In [ ]:
"""
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                  # load_model_path='../input/upto-e2/model_v1/checkpoint-2500/pytorch_model.bin'
                 )
                  """

In [ ]:
import time
df_test = pd.read_csv("./test.csv")
df_test.head()
abstracts = []
"""
device = torch.device("cuda")
model.to(device)
"""
model.eval();
with open("./predictions_2.csv",'w') as csv_file:
    csv_writer = csv.writer(csv_file,delimiter=',')
    csv_writer.writerow(['title','abstract'])
    start = time.time()
    for i in range(0,len(df_test)//100,10):
        if i%50==0:
            print(i,time.time()-start)
            start=time.time()
        title = df_test.iloc[i]['title']
        prompt = SPECIAL_TOKENS['bos_token'] + " " + title.strip() + " " + SPECIAL_TOKENS['sep_token']
                
        generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
        
        generated = generated.to(device)

        

        sample_output = model.generate(generated, 
                                        do_sample=True,   
                                        min_length=50, 
                                        max_length=MAXLEN,
                                        top_k=15,                                 
                                        top_p=0.9,        
                                        temperature=0.9,
                                        repetition_penalty=2.0,
                                        num_return_sequences=1
                                        )
        # print(sample_output)
        text = tokenizer.decode(sample_output[0], skip_special_tokens=True)
        abstract = text[len(title):]
        abstracts.append(abstract)
        csv_writer.writerow([title,abstract.strip()])
        # print(title,abstract)

In [ ]:
# torch.cuda.empty_cache()
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                  # load_model_path='../input/upto-e2/model_v1/checkpoint-2500/pytorch_model.bin'
                 )

In [ ]:
"""
import time
df_test = pd.read_csv("./test.csv")
df_test.head()
abstracts = []

device = torch.device("cuda")
model.to(device)

tokenizer.padding_side = 'left'
model.eval();
with open("./predictions.csv",'w') as csv_file:
    
    csv_writer = csv.writer(csv_file,delimiter=',')
    csv_writer.writerow(['title','abstract'])
    start = time.time()
    for i in range(0,len(df_test),32):
        print(i,time.time()-start)
        start=time.time()
        titles = []
        prompts = []
        for j in range(i,min(i+32,len(df_test))):
            title = df_test.iloc[j]['title'].strip()
            titles.append(title)
            prompt = SPECIAL_TOKENS['bos_token'] + " " + title + " " + SPECIAL_TOKENS['sep_token'] + " "
            prompts.append(prompt)

        encoded_prompt_dict = tokenizer.batch_encode_plus(prompts, return_tensors="pt", pad_to_max_length=True)
        encoded_prompt = encoded_prompt_dict['input_ids'].to(dev)
        encoded_mask = encoded_prompt_dict['attention_mask'].to(dev)
        
        ops = model.generate(encoded_prompt,
                                       attention_mask = encoded_mask,
                                        do_sample=True,   
                                        min_length=100, 
                                        max_length=MAXLEN,
                                        top_k=15,                                 
                                        top_p=0.9,        
                                        temperature=0.8,
                                        repetition_penalty=2.0,
                                        num_return_sequences=1,
                                        length_penalty=1.0
                                        )
        for title,op in zip(titles,ops):
            text = tokenizer.decode(op, skip_special_tokens=True)
            abstract = text[len(title):]
            csv_writer.writerow([title,abstract.strip()])
!python3 "../input/submission-script/submit_track2.py" "./predictions.csv"
#torch.cuda.empty_cache()
"""

In [ ]:
!head "./predictions.csv"

In [ ]:
!rm "./predictions.csv" "./submission.csv"

In [ ]:
#!pip install GPUtil

#import torch
#from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    #gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    #gpu_usage()

free_gpu_cache()      

In [ ]:
prompts = []
tokenizer.padding_side = 'left'
ml = 0
for i in range(64):
    title = df_test.iloc[i]['title'].strip(" ")
    ml = max(ml,len(title.split(" ")))
    prompt = SPECIAL_TOKENS['bos_token'] + " " + title.strip() + " " + SPECIAL_TOKENS['sep_token']
    #tokenizer.encode(prompt,padding_side='left')
    #generated = torch.tensor(tokenizer.encode(prompt,truncation=True, padding="max_length")).unsqueeze(0)
    #fin_generated.append(generated)
    prompts.append(prompt)

tokenizer.padding_side = "left"
encoded_prompt_dict = tokenizer.batch_encode_plus(prompts, return_tensors="pt", pad_to_max_length=True)
encoded_prompt = encoded_prompt_dict['input_ids'].to(dev)
encoded_mask = encoded_prompt_dict['attention_mask'].to(dev)
"""
generated_batch = []
for prompt in prompts:
    generated_batch.append(torch.tensor(tokenizer.encode(prompt,padding='max_length',max_length=ml)))
torch.stack(generated_batch)"""

In [ ]:
start = time.time()
ops = model.generate(encoded_prompt,
               attention_mask =  encoded_mask,
                                        do_sample=True,   
                                        min_length=50, 
                                        max_length=MAXLEN,
                                        top_k=15,                                 
                                        top_p=0.9,        
                                        temperature=0.9,
                                        repetition_penalty=2.0,
                                        num_return_sequences=1
                                        )
print(time.time() - start)

In [ ]:
!python3 "../input/submission-script/submit_track2.py" "./predictions.csv"

In [ ]:
"""

title = df.iloc[1]['title']

prompt = SPECIAL_TOKENS['bos_token'] + title + SPECIAL_TOKENS['sep_token']
         
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)
model.to(device)
model.eval();


# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                min_length=50, 
                                max_length=MAXLEN,
                                top_k=30,                                 
                                top_p=0.7,        
                                temperature=0.9,
                                repetition_penalty=2.0,
                                num_return_sequences=10
                                )

for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=True)
    a = len(title) # + len(','.join(keywords))    
    print("{}: {}\n\n".format(i+1,  text[a:]))"""